In [1]:
from bs4 import BeautifulSoup
import re
from nltk import tokenize
import pandas as pd
from tqdm import tqdm

In [2]:
# download pmc xml using the following query:
# https://www.ncbi.nlm.nih.gov/pmc/?term=synthetic+dataset%5BBody+-+All+Words%5D
# and load it via bs4 (it takes a while)

file = 'pmc_result.xml'
infile = open(file, 'r', encoding='utf-8')
contents = infile.read()
soup = BeautifulSoup(contents, 'lxml-xml')
articles = soup.select('article')

In [ ]:
# parse the xml keeping the doi and the sentences that contain both a url and the word 'sythetic'

lst = []
for article in tqdm(articles):
    for i in article.select('article-meta'):
        input_tag = i.find(attrs={"pub-id-type" : "doi"})
        try:
            doi = input_tag.text
        except:
            doi = 'NA'
    for body in article.select('body'):
        for paragraph in body.select('p'):
            for sentence in tokenize.sent_tokenize(paragraph.text):
                ur = re.findall('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\), ]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', sentence)
                if ur:
                    for match in re.finditer(r"synthetic", sentence, re.IGNORECASE):
                        lst.append([doi,str(sentence)])

# save dataframe to file
df = pd.DataFrame.from_records(lst).drop_duplicates()
df.columns = ['doi','sentence']
df.to_csv('pmc_synth_datasets.csv',index=None)

 73%|███████▎  | 1434/1964 [02:07<01:21,  6.49it/s]